# Deep MNIST for Experts

Tensor Flowのチュートリアル「Deep MNIST for Experts」に沿って解説します。<br>
<a href="https://www.tensorflow.org/versions/r0.10/tutorials/index.html">View Tutrial</a>

## 前回の復習

Tensor Flowのチュートリアル「MNIST For ML Beginners」を簡単におさらい。<br>
<a href="https://github.com/e-xample/tensorflow/blob/master/MNIST_For_ML_Beginners.ipynb">詳しくはこちら</a>

ニューラルネットワークのモデル構築のおおまかな流れ、覚えていますか？

+ 学習データ・テストデータの準備
+ 入出力の設計
+ 入力層-出力層間のユニット結合の構造設計
+ パラメータ(重みW, バイアスb)の初期化
+ 出力層の出力定義
+ 損失関数の定義
+ 最適化アルゴリズムの定義
+ 評価関数の定義
+ パラメータの学習
+ 評価

In [5]:
import tensorflow as tf

# 学習データ・テストデータの準備(手書き文字のロード)
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

sess = tf.InteractiveSession()

# 入出力の設計(入力784次元、出力10次元)
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

# 入力層-出力層間のユニット結合の構造設計(入力層、出力層の２層構造。Fully-Connected。)
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

# パラメータ(重みW, バイアスb)の初期化
sess.run(tf.initialize_all_variables())

# 出力層の出力定義(出力層の活性化関数として、softmax関数を用いている)
y = tf.nn.softmax(tf.matmul(x,W) + b)

# 損失関数の定義(損失関数として、cross-entopy関数を用いている)
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

# 最適化アルゴリズムの定義(最適化アルゴリズムとして、勾配降下法を用いている)
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# 評価関数の定義(正解率の表示)
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# パラメータ(重みW, バイアスb)の学習(学習方法として、確率的勾配降下法を用いている)
for i in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})
    
# 評価
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Exception ignored in: <bound method BaseSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x10a2ee9b0>>
Traceback (most recent call last):
  File "/Users/a14197/.pyenv/versions/3.5.2/lib/python3.5/site-packages/tensorflow/python/client/session.py", line 522, in __del__
    self.close()
  File "/Users/a14197/.pyenv/versions/3.5.2/lib/python3.5/site-packages/tensorflow/python/client/session.py", line 1262, in close
    self._default_session.__exit__(None, None, None)
  File "/Users/a14197/.pyenv/versions/3.5.2/lib/python3.5/contextlib.py", line 66, in __exit__
    next(self.gen)
  File "/Users/a14197/.pyenv/versions/3.5.2/lib/python3.5/site-packages/tensorflow/python/framework/ops.py", line 3536, in get_controller
    % type(default))
AssertionError: Nesting violated for default stack of <class 'weakref'> objects


0.9186


<img src="Deep_MNIST_for_Experts/perceptron.jpg">

「MNIST For ML Beginners」では、入力層と出力層の２層のみからなり、層間は全結合という簡単な構造を扱いました。  
今回は「画像認識」というタスクに対する適切なニューラルネットワークモデルを構築してみましょう！

## 畳み込みニューラルネットワーク(Convolutional Neural Network)

<img src="Deep_MNIST_for_Experts/CNN.jpg">
今回は畳み込みニューラルネットワーク(以下、CNN)と呼ばれる構造のモデルを扱って手書き文字の認識を行います。  
CNNは、一般的に後ほど説明する「畳み込み層」と「プーリング層」を何層か繰り返した後、全結合した「多層パーセプトロン」が配置される構成になっています。  
脳の視覚野における神経科学の知見を基に開発されたモデルで、画像の特徴をうまく学習（認識）する構成として画像認識の分野で注目されています。

### 入出力について
「MNIST For ML Beginners」では、画像の列ベクトルに変換して入力としていました。  
CNNでは、画像の２次元的な位置関係の情報を保存したまま学習するために、入力画像の各画素の符号化した2次元行列を入力とします。  
出力は、前回同様も10クラスの分類を行うので、各クラスに分類される確率分布を出力とします。

### 入力層-出力層間のユニット結合の構造について

#### 畳み込み層(Convolution Layer)
入力された画像に、小サイズの二次元フィルタを畳み込む演算を行うことで、画像の持つ局所的な特徴を抽出する役割を果たす層です。  
畳み込む演算とは、二次元フィルタに割り当てられた重みとバイアスと入力の積和をとる演算。  
この二次元フィルタを画像内の各位置に移動させて同様の演算を適用し、結果として得られる２次元の出力を畳み込み層の出力とします。  
このフィルタにかけた出力画像は特徴マップ(Feature Map)と呼ばれています。  

また、二次元フィルタは異なる複数枚用意され、二次元フィルタごとに特徴マップを得ます。  
フィルタを増やすことで入力画像のさまざまな特徴を捉えられるようになります。

CNNのモデルの特徴は、このフィルタを学習によって自動獲得できるという点です。  
最初、二次元フィルタの重み・バイアスにはランダムな値が入っていてよくわからない特徴にしか反応しないが、学習が進むと縦線や横線など画像認識に重要な特徴に強く反応するようになっていくようなフィルタが得られます。  
これがDeep Learningの力の源で、(特徴)表現学習と呼ばれています。  
すなわち、畳み込み層の二次元フィルタの重み・バイアスが、CNNにおける更新（学習）の対象となるパラメータとなります。

　　　　　　　　　　　　[学習前]
<img src="Deep_MNIST_for_Experts/weight_before.png">

　　　　　　　　　　　　[学習後]
<img src="Deep_MNIST_for_Experts/weight_after.png">

#### プーリング層(Pooling Layer)
入力された画像の小領域での値を一つの値に集約し解像度を落とすことで、抽出された特徴の位置が若干変化しても取り出される特徴はほとんど変化しないという特徴の不変性を獲得する役割を果たす層である。つまり、表記ブレに対して頑健になるという効果があります。  
プーリング層での演算は、入力された画像の小領域の最大値を出力するMax Poolingや、小領域の平均値を出力するMean Poolingなどがありますが、Max Poolingが使われることが多いようです。
プーリング層には、重みやバイアスはないため、プーリング層の出力に学習は影響しません。

<img src="Deep_MNIST_for_Experts/conv-pool.png">

#### 多層パーセプトロン(Full-connected Layer)
最後に、畳み込み層やプーリング層によって得られた特徴マップを多層パーセプトロンの入力とし、多層パーセプトロン層の重み・バイアスを通して最終的に出力を得ます。 
多層パーセプトロン層の重み・バイアスも、CNNにおける更新（学習）の対象となるパラメータです。

以上が、CNN全体のモデルの構造です。  

## 実装しよう！

まずは重みやバイアスを初期化する関数を定義しておきます。  
今回は、単純な乱数で初期化するのではなく、（ある）平均値と分散に基づいて初期化されます。  
なお、バイアスはのフィルター内で一定の値とすることが一般的です。

In [6]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

畳み込み層とプーリング層の演算処理も関数として定義しておきます。  

In [7]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

### 入出力の設計(入力28*28次元、出力10次元)

In [8]:
x_image = tf.reshape(x, [-1,28,28,1])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

### 入力層-出力層間のユニット結合の構造設計

今回は、活性化関数としてReLU関数を利用しています。  
出力層以外の活性化関数として、よく利用される関数です。  
sigmoid関数などの他の有名な活性化関数と比較して、学習の収束が早いことや勾配消失問題を解決しやすくなるなどといったメリットがあります。

In [9]:
# 第1畳み込み層 (フィルターサイズ: 5*5、入力: 1チャネル、フィルター枚数: 32枚)
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

# 第1プーリング層 (フィルターサイズ: 2*2)
h_pool1 = max_pool_2x2(h_conv1)

In [10]:
# 第2畳み込み層 (フィルターサイズ: 5*5、入力: 32チャネル、フィルター枚数: 64枚)
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

# 第2プーリング層 (フィルターサイズ: 2*2)
h_pool2 = max_pool_2x2(h_conv2)

In [11]:
# 全結合多層パーセプトロン
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# 全結合層から出力層への結合
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

### 出力層の出力定義
前回同様、出力層の活性化関数として、softmax関数を用いています。  
今回はドロップアウトという手法を導入することにより、局所解への収束を防いでいます。  
https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf

In [13]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

### 損失関数の定義
前回同様、損失関数として、cross-entopy関数を用いています。

In [14]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))

### 最適化アルゴリズムの定義

今回はAdamと呼ばれる最適化アルゴリズムを利用している。  
https://arxiv.org/pdf/1412.6980.pdf

In [ ]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

### 評価関数の定義

In [ ]:
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

### パラメータの学習

学習経過も出力している。

In [ ]:
sess.run(tf.initialize_all_variables())
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

### 評価

In [15]:
print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.1
step 100, training accuracy 0.84
step 200, training accuracy 0.8
step 300, training accuracy 0.86
step 400, training accuracy 0.92
step 500, training accuracy 1
step 600, training accuracy 0.94
step 700, training accuracy 0.96
step 800, training accuracy 0.96
step 900, training accuracy 0.96
step 1000, training accuracy 0.96
step 1100, training accuracy 0.98
step 1200, training accuracy 0.94
step 1300, training accuracy 1
step 1400, training accuracy 1
step 1500, training accuracy 0.96
step 1600, training accuracy 0.98
step 1700, training accuracy 1
step 1800, training accuracy 1
step 1900, training accuracy 0.98
step 2000, training accuracy 0.98
step 2100, training accuracy 1
step 2200, training accuracy 1
step 2300, training accuracy 0.96
step 2400, training accuracy 0.98
step 2500, training accuracy 1
step 2600, training accuracy 0.98
step 2700, training accuracy 1
step 2800, training accuracy 1
step 2900, training accuracy 1
step 3000, training accurac

# 99.2%おおおおおおおおお！